<a href="https://colab.research.google.com/github/Kevin27Sanchez/Modelo-TestGen/blob/main/Parte1/TESTGEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets seqeval fsspec==2024.10.0
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#2. CARGAR EL DATASET DESDE EL DRIVE
import pandas as pd
import re
from datasets import Dataset
from transformers import BertTokenizerFast
from google.colab import drive

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/TESIS/REQUISITOS_A_PRUEBAS.xlsx'
df = pd.read_excel(file_path)

print(df.head())


Mounted at /content/drive
   N°                                          Requisito  \
0   1  Tipo: Cambio de requisito\n\nNombre: Agregar f...   
1   2  NOMBRE\nReporte de Notas de débito\n\nDESCRIPC...   
2   3  NOMBRE\nReporte de Notas de crédito\n\nDESCRIP...   
3   4  1- TIPO: Nuevo requisito\n\n2- NOMBRE\nReporte...   
4   5  - La compra al crédito tiene que activarse med...   

                                          Extracción  
0  El lìmite máximo de selección de familia debe ...  
1  Al seleccionar el rango de fecha sólo debe pod...  
2  Al seleccionar el rango de fecha sólo debe pod...  
3  El usuario deberá seleccionar el rango de fech...  
4  Cuando la cuota es mayor a 1, el día de pago t...  


In [ ]:
#3. LIMPIAR EL TEXTO DE LOS REQUISITOS
def limpiar_texto(texto):
    texto_limpio = re.sub(r'\n+', ' ', texto)
    texto_limpio = re.sub(r'\s+', ' ', texto).strip()
    return texto_limpio

file_path = '/content/drive/MyDrive/TESIS/REQUISITOS_A_PRUEBAS.xlsx'
df = pd.read_excel(file_path)

if 'Requisito' in df.columns and 'Extracción' in df.columns:
    df['Requisito'] = df['Requisito'].fillna('').apply(limpiar_texto)
    df['Extracción'] = df['Extracción'].fillna('').apply(limpiar_texto)
else:
    print("Error: Las columnas 'Requisito' y/o 'Extracción' no existen en el DataFrame.")

print(df.head())


   N°                                          Requisito  \
0   1  Tipo: Cambio de requisito Nombre: Agregar filt...   
1   2  NOMBRE Reporte de Notas de débito DESCRIPCIÓN ...   
2   3  NOMBRE Reporte de Notas de crédito DESCRIPCIÓN...   
3   4  1- TIPO: Nuevo requisito 2- NOMBRE Reporte de ...   
4   5  - La compra al crédito tiene que activarse med...   

                                          Extracción  
0  El lìmite máximo de selección de familia debe ...  
1  Al seleccionar el rango de fecha sólo debe pod...  
2  Al seleccionar el rango de fecha sólo debe pod...  
3  El usuario deberá seleccionar el rango de fech...  
4  Cuando la cuota es mayor a 1, el día de pago t...  


In [ ]:
#4. Inicializar el tokenizador rápido de BERT
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Crear un diccionario de mapeo para las etiquetas BIO
label_map = {'O': 0, 'B-key': 1, 'I-key': 2}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
#5. Función para convertir etiquetas de texto a etiquetas numéricas
def convert_labels_to_ids(labels):
    return [label_map[label] if label in label_map else -100 for label in labels]


In [ ]:
# 6. Función para crear etiquetas BIO
def create_bio_labels(requisito, extraccion, tokens):
    labels = ['O'] * len(tokens)


    extraccion_tokens = tokenizer.tokenize(extraccion)
    extraccion_length = len(extraccion_tokens)

    for i in range(len(tokens)):
        if tokens[i:i+extraccion_length] == extraccion_tokens:
            labels[i] = 'B-key'
            for j in range(1, extraccion_length):
                labels[i+j] = 'I-key'
            break

    return labels


In [ ]:
# 7. VENTANAS DESLIZANTES

window_size = 512
stride = 256

all_input_ids = []
all_attention_masks = []
all_labels = []

for i, row in df.iterrows():
    text = row['Requisito']
    extraccion = row['Extracción']

    tokens = tokenizer.tokenize(text)

    bio_labels = create_bio_labels(text, extraccion, tokens)

    for start_idx in range(0, len(tokens), stride):
        end_idx = min(start_idx + window_size, len(tokens))
        window_tokens = tokens[start_idx:end_idx]
        window_labels = bio_labels[start_idx:end_idx]

        if len(window_tokens) < window_size:
            pad_length = window_size - len(window_tokens)
            window_tokens += ['[PAD]'] * pad_length
            window_labels += ['O'] * pad_length

        input_ids = tokenizer.convert_tokens_to_ids(window_tokens)

        attention_mask = [1 if token != '[PAD]' else 0 for token in window_tokens]

        window_labels_ids = convert_labels_to_ids(window_labels)

        if len(window_tokens) < window_size:
            window_labels_ids[-pad_length:] = [-100] * pad_length

        assert len(input_ids) == 512, f"Error: secuencia demasiado larga ({len(input_ids)} tokens)"

        all_input_ids.append(input_ids)
        all_attention_masks.append(attention_mask)
        all_labels.append(window_labels_ids)

new_df = pd.DataFrame({
    'input_ids': all_input_ids,
    'attention_mask': all_attention_masks,
    'labels': all_labels
})

tokenized_dataset = Dataset.from_pandas(new_df)

print(tokenized_dataset)


Token indices sequence length is longer than the specified maximum sequence length for this model (649 > 512). Running this sequence through the model will result in indexing errors


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 62
})


In [ ]:
# Verificar la longitud de las secuencias tokenizadas
for i, input_id in enumerate(tokenized_dataset['input_ids']):
    print(f"Longitud de la secuencia {i}: {len(input_id)} tokens")


Longitud de la secuencia 0: 512 tokens
Longitud de la secuencia 1: 512 tokens
Longitud de la secuencia 2: 512 tokens
Longitud de la secuencia 3: 512 tokens
Longitud de la secuencia 4: 512 tokens
Longitud de la secuencia 5: 512 tokens
Longitud de la secuencia 6: 512 tokens
Longitud de la secuencia 7: 512 tokens
Longitud de la secuencia 8: 512 tokens
Longitud de la secuencia 9: 512 tokens
Longitud de la secuencia 10: 512 tokens
Longitud de la secuencia 11: 512 tokens
Longitud de la secuencia 12: 512 tokens
Longitud de la secuencia 13: 512 tokens
Longitud de la secuencia 14: 512 tokens
Longitud de la secuencia 15: 512 tokens
Longitud de la secuencia 16: 512 tokens
Longitud de la secuencia 17: 512 tokens
Longitud de la secuencia 18: 512 tokens
Longitud de la secuencia 19: 512 tokens
Longitud de la secuencia 20: 512 tokens
Longitud de la secuencia 21: 512 tokens
Longitud de la secuencia 22: 512 tokens
Longitud de la secuencia 23: 512 tokens
Longitud de la secuencia 24: 512 tokens
Longitud d

In [ ]:
#8. ENTRENAMIENTO DEL MODELO
import os
from sklearn.model_selection import train_test_split
from transformers import BertForTokenClassification, Trainer, TrainingArguments

os.environ["WANDB_DISABLED"] = "true"

train_df, eval_df = train_test_split(new_df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=3)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,No log,0.076682
2,No log,0.070623
3,No log,0.073299


TrainOutput(global_step=21, training_loss=0.18048646336510069, metrics={'train_runtime': 997.3601, 'train_samples_per_second': 0.147, 'train_steps_per_second': 0.021, 'total_flos': 38410970508288.0, 'train_loss': 0.18048646336510069, 'epoch': 3.0})

In [ ]:
from seqeval.metrics import classification_report

predictions, labels, _ = trainer.predict(eval_dataset)
predictions = predictions.argmax(axis=2)

true_labels = []
predicted_labels = []

for i in range(len(labels)):
    true_label = []
    predicted_label = []
    for j in range(len(labels[i])):
        if labels[i][j] != -100:
            true_label.append([k for k, v in label_map.items() if v == labels[i][j]][0])
            predicted_label.append([k for k, v in label_map.items() if v == predictions[i][j]][0])
    true_labels.append(true_label)
    predicted_labels.append(predicted_label)

print(classification_report(true_labels, predicted_labels))


              precision    recall  f1-score   support

         key       0.00      0.00      0.00         2

   micro avg       0.00      0.00      0.00         2
   macro avg       0.00      0.00      0.00         2
weighted avg       0.00      0.00      0.00         2



/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
